In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

from pkg.global_config import RNB_PLANNING_DIR
from pkg.utils.utils import *    
from pkg.controller.combined_robot import *
from pkg.project_config import *

In [2]:
CONNECT_INDY = False

MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

In [3]:
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                MOBILE_IP),
    RobotConfig(1, RobotType.indy7, ((0.172,0,0.439), (0,0,np.pi)),
                INDY_IP, root_on="kmb0_platform", specs={"no_sdk":True})]
              , connection_list=[False, CONNECT_INDY])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
# xyz_rpy_robots = {"kmb0": ((0,0,0), (0,0,0)), "indy1": ((0,0,0), (0,0,np.pi))}
# crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gscene.show_pose(-crob.home_pose)

connection command:
kmb0: False
indy1: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [4]:
# gscene.show_pose(-crob.home_pose + ([0.2,0.2,np.pi/3,]+[0]*6))

## set workspace

In [5]:
gscene.set_workspace_boundary( -5, 5, -4, 4, -0.1, 1.75)

Please create a subscriber to the marker


## define robot and planning scene

In [6]:
from pkg.planning.scene import *
from demo_utils.environment import *

CHAIR_NAME = "chair0"
CHAIR_DIM = (0.4,0.4,0.5)

MOBILE_NAME = "kmb0"
HOLD_LINK = "kmb0_platform"
HOLD_XYZ = (0.445 + 0.35, 0, CHAIR_DIM[2]/2)
HOLD_RPY = (0, -np.pi/2, -np.pi/2)
HOLD_NAME = "hold0"

ROBOT_NAME = "indy1"
TOOL_LINK = "indy1_tcp"
TOOL_NAME = "brush_face"

HOME_POSE = -crob.home_pose
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)

gtems_robot = s_builder.add_robot_geometries(color=(0, 1, 0, 0.5), display=True, collision=True)
viewpoint = add_cam(gscene, tool_link=TOOL_LINK)
add_indy_tool_kiro(gscene, tool_link=TOOL_LINK, face_name=TOOL_NAME, zoff=-0.04)

pscene = PlanningScene(gscene, combined_robot=crob)

## planning pipeline

In [7]:

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)


from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.motion_filters = [GraspChecker(pscene)]

 * Environment: production
   Use a production WSGI server instead.


## Add action points

In [9]:
from pkg.planning.constraint.constraint_common import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

 * Debug mode: off


In [10]:
CLEARANCE = 0.001
# floor
gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link",
                   dims=(10,8,0.1), center=(0,0,-0.05-CLEARANCE), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)
floor = pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)

# table
gscene.create_safe(gtype=GEOTYPE.BOX, name="table", link_name="base_link",
                   dims=(2,0.7,0.5), center=(2,0,0.25), 
                   rpy=(0,0,0), color=(0.9, 0.9, 0.9, 0.9), display=True,
                   collision=True, fixed=True)

# brush_face = pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, 
#                                   point=(-gscene.NAME_DICT['brush_face'].dims[0]/2,0,0), rpy=(0,np.pi/2*1,0))

# waypoint
WP_DIMS = (0.6,0.4,0.1)
gscene.create_safe(gtype=GEOTYPE.BOX, name="wayframer", link_name=HOLD_LINK,
                   dims=WP_DIMS, center=(0,0,WP_DIMS[2]/2), rpy=(0,0,0), color=(1, 0, 0, 0.5), display=True,
                   collision=False, fixed=True)
wayframer = pscene.create_binder(bname="wayframer", gname="wayframer", _type=WayFramer, 
                                 point=(0,0,-WP_DIMS[2]/2), rpy=(0,0,0))


waypoints = [((2-0.5,-1,0),(0,0,np.pi/2)), ((2+0.5,-1,0),(0,0,np.pi/2)), ((2+0.5,1,0),(0,0,-np.pi/2)), ((2-0.5,1,0),(0,0,-np.pi/2))]
wp_list = []
for i_w, (xyz, rpy) in enumerate(waypoints):
    wp_list.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="wp_{}".format(i_w), link_name="base_link",
                           dims=WP_DIMS, center=xyz, 
                           rpy=rpy, color=(0.6, 0.0, 0.0, 0.5), display=True, collision=False, fixed=True, parent="floor")
    )
wp_task = pscene.create_subject(oname="waypoints", gname="floor", _type=WayopintTask, 
                                action_points_dict={wp.name: WayFrame(wp.name, wp, [0,0,0.05], [0,0,0]) for wp in wp_list})


# chair holder
gscene.create_safe(gtype=GEOTYPE.SPHERE, name=HOLD_NAME, link_name=HOLD_LINK,
                   dims=(0.01,) * 3, center=HOLD_XYZ, rpy=HOLD_RPY, color=(1, 0, 0, 0.5), display=True,
                   collision=False,
                   fixed=True)
holder = pscene.create_binder(bname=HOLD_NAME, gname=HOLD_NAME, _type=Gripper2Tool, point=(0, 0, 0), rpy=(0, 0, 0))
holder.redundancy = {}

## Add chair

In [11]:
chair_loc = (1, -1, CHAIR_DIM[2]/2)

obs_all = 3
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name=CHAIR_NAME, link_name="base_link",
                   dims=CHAIR_DIM, center=chair_loc, rpy=(0,0,0),
                   color=(1,1,0,1), display=True, collision=True, fixed=False)
chair = pscene.create_subject(oname=CHAIR_NAME, gname=CHAIR_NAME, _type=CylinderObject, 
                              GRASP_WIDTH_MIN=CHAIR_DIM[0]-0.1, GRASP_WIDTH_MAX=CHAIR_DIM[0]+0.1, 
                              GRASP_DEPTH_MIN=CHAIR_DIM[0]/2, GRASP_DEPTH_MAX=CHAIR_DIM[0]/2)
chair.action_points_dict[CHAIR_NAME+"_side_g"].redundancy = {'u': (-np.pi, np.pi)}

## [Test] move chair to cover waypoints

In [12]:
initial_state = pscene.initialize_state(-crob.home_pose)
goal_nodes = [('floor',)+(4,)]

In [33]:
from pkg.planning.task.rrt_star import TaskRRTstar
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler


tplan = TaskRRTstar(pscene)
tplan.REWIND_MAX = 3
tplan.new_node_sampler = PenaltyNodeSampler(3, 3)
tplan.parent_node_sampler = UniformNodeSampler(3)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

mplan.reset_log(True)

ppline.search(initial_state, goal_nodes, verbose=True,
              timeout=0.5, timeout_loop=5, multiprocess=False,
              add_homing=3, home_pose=HOME_POSE, terminate_on_first=False)
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_home = snode_schedule+ppline.add_return_motion(snode_schedule[-1], try_count=3, initial_state=initial_state)
ppline.play_schedule(snode_schedule_home)


try: 0 - ('floor', 0)->('floor', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('hold0', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('hold0', 0) = success
branching: 0->1 (0.1/5.0 s, steps/err: 54(94.2778587341 ms)/0.00216936214447)
try: 1 - ('hold0', 0)->('hold0', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('hold0', 0)->('hold0', 1) = fail
try: 1 - ('hold0', 0)->('hold0', 1)
Motion Filer Failure: GraspChecker
result: 1 - ('hold0', 0)->('hold0', 1) = fail
try: 1 - ('hold0', 0)->('floor', 0)
try transition motion
transition motion tried: True
result: 1 - ('hold0', 0)->('floor', 0) = success
branching: 1->2 (0.17/5.0 s, steps/err: 42(61.6700649261 ms)/0.000676165672198)
try: 2 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 2 - ('floor', 0)->('floor', 1) = success
branching: 2->3 (0.28/5.0 s, steps/err: 47(102.596998215 ms)/0.00138880878694)
t

transition motion tried: True
reserve 23 -> 14
result: 22 - ('floor', 0)->('floor', 1) = success
branching: 22->23 (2.11/5.0 s, steps/err: 68(93.0738449097 ms)/0.0011499285499)
try: 23 - ('floor', 1)->('floor', 2)
try transition motion
transition motion tried: True
reserve 24 -> 10
reserve 24 -> 15
result: 23 - ('floor', 1)->('floor', 2) = success
branching: 23->24 (2.16/5.0 s, steps/err: 9(49.7481822968 ms)/0.00128041819466)
try: 24 - ('floor', 2)->('floor', 3)
try transition motion
transition motion tried: True
result: 24 - ('floor', 2)->('floor', 3) = success
branching: 24->25 (2.29/5.0 s, steps/err: 82(131.675004959 ms)/0.0013696710899)
try: 24 - ('floor', 2)->('floor', 3)
try transition motion
transition motion tried: True
result: 24 - ('floor', 2)->('floor', 3) = success
branching: 24->26 (2.4/5.0 s, steps/err: 81(105.46207428 ms)/0.00163815408942)
try: 25 - ('floor', 3)->('floor', 4)
try transition motion
transition motion tried: True
result: 25 - ('floor', 3)->('floor', 4) = su

result: 37 - ('floor', 3)->('floor', 4) = success
branching: 37->46 (4.96/5.0 s, steps/err: 9(70.9829330444 ms)/0.00167564518259)
try: 38 - ('floor', 3)->('floor', 4)
try transition motion
transition motion tried: True
result: 38 - ('floor', 3)->('floor', 4) = success
branching: 38->47 (5.01/5.0 s, steps/err: 9(52.3540973663 ms)/0.00109859753359)
======================= terminated 0: max iteration time reached (1630530433/1630530428.51 s) ===============================
('floor', 0)->('floor', 0)
('floor', 0)->('hold0', 0)
('hold0', 0)->('floor', 0)
('floor', 0)->('floor', 1)
('floor', 1)->('floor', 2)
('floor', 2)->('hold0', 2)
('hold0', 2)->('floor', 2)
('floor', 2)->('floor', 3)
('floor', 3)->('floor', 4)
('floor', 4)->('floor', 4)


In [34]:
schedules

{13: [0, 1, 2, 3, 4, 5, 8, 10, 11, 12, 13],
 18: [0, 1, 2, 3, 4, 7, 14, 16, 18],
 21: [0, 1, 2, 3, 4, 7, 14, 15, 17, 20, 21],
 27: [0, 1, 2, 19, 22, 23, 24, 25, 27],
 28: [0, 1, 2, 19, 22, 23, 24, 26, 28],
 30: [0, 1, 2, 3, 4, 5, 6, 29, 30],
 45: [0, 1, 2, 3, 9, 32, 33, 36, 45],
 46: [0, 1, 2, 3, 9, 32, 33, 37, 46],
 47: [0, 1, 2, 3, 9, 32, 33, 38, 47]}

In [35]:
schedules_sorted

[[0, 1, 2, 3, 4, 7, 14, 16, 18],
 [0, 1, 2, 19, 22, 23, 24, 26, 28],
 [0, 1, 2, 19, 22, 23, 24, 25, 27],
 [0, 1, 2, 3, 9, 32, 33, 38, 47],
 [0, 1, 2, 3, 4, 5, 6, 29, 30],
 [0, 1, 2, 3, 9, 32, 33, 36, 45],
 [0, 1, 2, 3, 9, 32, 33, 37, 46],
 [0, 1, 2, 3, 4, 7, 14, 15, 17, 20, 21],
 [0, 1, 2, 3, 4, 5, 8, 10, 11, 12, 13]]

In [15]:
ppline.play_schedule(snode_schedule_home)

('floor', 0)->('floor', 0)
('floor', 0)->('hold0', 0)
('hold0', 0)->('floor', 0)
('floor', 0)->('floor', 1)
('floor', 1)->('floor', 2)
('floor', 2)->('floor', 3)
('floor', 3)->('floor', 4)
('floor', 4)->('floor', 4)


In [17]:
hash_list = []
hash_list.append(make_state_param_hashable(pscene, tplan.snode_dict[3].state, as_dict=False))
hash_list.append(make_state_param_hashable(pscene, tplan.snode_dict[22].state, as_dict=False))
hash_list.append(make_state_param_hashable(pscene, tplan.snode_dict[25].state, as_dict=False))

In [20]:
hset = set()
for hv in hash_list:
    if hv not in hset:
        hset.add(hv)
print(len(hset))

3


3

In [19]:
tplan.neighbor_nodes

{('floor', 0): None,
 ('floor', 1): None,
 ('floor', 2): None,
 ('floor', 3): None,
 ('hold0', 0): None,
 ('hold0', 1): None,
 ('hold0', 2): None,
 ('hold0', 3): None,
 ('hold0', 4): None}

In [25]:
for node, sid_list in sorted(tplan.node_snode_dict.items()):
    print("node: {}".format(node))
    print(",".join(["{:3}, ".format(sidx) for sidx in sid_list]))
    print(",".join(["{:3}, ".format(tplan.snode_dict[sidx].depth) for sidx in sid_list]))

node: ('floor', 0)
  0, ,  2, 
  0, ,  2, 
node: ('floor', 1)
  3, 
  3, 
node: ('floor', 2)
  7, , 16, , 21, , 32, , 33, , 58, 
  4, ,  6, ,  6, ,  6, ,  6, ,  4, 
node: ('floor', 3)
  8, , 10, , 17, , 22, , 23, , 34, , 35, , 36, , 37, , 59, , 60, , 61, , 62, , 63, , 64, 
  5, ,  5, ,  7, ,  7, ,  7, ,  7, ,  7, ,  7, ,  7, ,  5, ,  5, ,  5, ,  5, ,  5, ,  5, 
node: ('floor', 4)
  9, , 11, , 12, , 18, , 20, , 24, , 25, , 26, , 27, , 28, , 29, , 38, , 39, , 40, , 41, , 43, , 44, , 45, , 46, , 48, , 49, , 50, , 51, , 53, , 54, , 55, , 56, , 67, , 68, , 69, , 70, , 71, , 72, , 73, , 81, , 82, , 83, , 84, , 85, , 86, , 87, , 95, , 96, , 97, , 98, , 99, ,100, ,101, ,109, ,110, ,111, ,112, ,113, ,114, ,115, ,123, 
  6, ,  6, ,  6, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  8, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, ,  6, 

In [25]:
# from pkg.planning.pddlstream.plan_rnb import solve_in_pddlstream

# goal_pairs=[(CHAIR_NAME, 'goal'), (CHAIR_NAME[:-1]+"1", "floor"), (CHAIR_NAME[:-1]+"2", "floor")]
# TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS = 1, 100, 100, 30
# GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO = 100, 100, False, 100


# res, plan, log_dict = solve_in_pddlstream(pscene, mplan, MOBILE_NAME, HOLD_NAME, HOME_POSE, goal_pairs,
#                         TIMEOUT_MOTION, MAX_TIME, MAX_ITER, MAX_SKELETONS,
#                         GRASP_SAMPLE, STABLE_SAMPLE, SHOW_STATE, SEARCH_SAMPLE_RATIO,
#                         use_pybullet_gui=False, USE_MOVEIT_IK=True)